# JaegerTracesIngest

<div>Establish a connection with the <u>gRPC API v2</u> of the Jaeger Instance instance installed in the namespace (reachable via the service <code>jaeger-query</code> on port <code>16686</code> behind the path <code>/jaeger</code>).</div><div><br></div><div>Some useful links:<br></div><div><ul><li><b>A brief overview of Jaeger APIs:</b> [https://www.jaegertracing.io/docs/1.18/apis/#grpcprotobuf-stable]</li><li><b>Jaeger gRPC API Proto Definition</b>: [https://github.com/jaegertracing/jaeger-idl/blob/master/proto/api_v2/query.proto]</li><li><b>The Java library for Jaeger Data Analytics</b>: [https://github.com/jaegertracing/jaeger-analytics-java]</li><li><b>Reference Jupyter Notebook</b>: [https://github.com/jaegertracing/jaeger-analytics-java/blob/master/jupyter/jaeger-query.ipynb]<br></li></ul></div>

We create a gRPC channel with Jaeger API v2 and create a **Trace Request** on the root service that will be run in the rest of the Notebook.<br>

In [2]:
import com.google.protobuf.ByteString
import io.grpc.ManagedChannel
import io.grpc.ManagedChannelBuilder
import io.jaegertracing.api_v2.Query.FindTracesRequest
import io.jaegertracing.api_v2.Query.GetTraceRequest
import io.jaegertracing.api_v2.Query.TraceQueryParameters
import io.jaegertracing.api_v2.QueryServiceGrpc
import io.jaegertracing.api_v2.QueryServiceGrpc.QueryServiceBlockingStub

val queryHostPort = "jaeger-query.monitoring.svc.cluster.local:16686/jaeger"
val appRootSvc = "jaeger-query"

val channel = ManagedChannelBuilder
                .forTarget(queryHostPort)
                .usePlaintext
                .build

val queryService = QueryServiceGrpc.newBlockingStub(channel)

val query = TraceQueryParameters
                .newBuilder
                .setServiceName(appRootSvc)
                .build

val jaegerTraceRequest = FindTracesRequest
                .newBuilder
                .setQuery(query)
                .build


Then we execute the query and get raw data, this raw data is mapped to Span objects from the model generated by protobuff.tbr>

In [4]:
import io.jaegertracing.api_v2.Model.Span
import io.jaegertracing.api_v2.Query.SpansResponseChunk

import collection.JavaConverters._

val spanIterator: Iterator[Span] = for { 
    spanChunks <- queryService.findTraces(jaegerTraceRequest).asScala
    span       <- spanChunks.getSpansList.asScala
} yield span

val spanList: List[Span] = spanIterator.toList


The list of Spans returned by the API is processed by Jaeger Analytics to return a *Trace () *and add custom behavior.<br>

In [6]:
import io.jaegertracing.analytics._
import io.jaegertracing.analytics.NetworkLatency.Name
import io.jaegertracing.analytics.gremlin.GraphCreator
import io.jaegertracing.analytics.model.Converter
import io.jaegertracing.analytics.model.Trace
import io.opentracing.tag.Tags
import org.apache.tinkerpop.gremlin.structure.Graph

// Shall not be called "Trace" imho but a collection of spans related to many traces (with multiple traceIds)
val trace: Trace = Converter.toModel(spanList.asJava)

for {
    span  <- trace.spans.asScala
    (k,v) <- span.tags.asScala
} println(span.operationName, span.spanId, k, v)
